In [1]:
%cd /content/drive/MyDrive/Kirill/Projects/Test task_VK_2

/content/drive/MyDrive/Kirill/Projects/Test task_VK_2


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae

In [4]:
train_df = pd.read_csv('data_preprocessing/final_train.csv')

X = train_df.loc[:, ~train_df.columns.isin(['score'])]
y = train_df['score']

In [5]:
X.head()

,lat,lon,0,7,8,10,11,12,18,21,...,92,93,107,109,159,242,250,346,347,distance
0,0.724309,0.210423,0.142355,0.291666,0.112751,0.064546,0.354221,0.938658,0.326386,0.440000,...,0.343240,0.411185,0.071107,0.165290,0.035966,0.838258,0.418502,0.039461,0.051389,0.080668
1,0.757334,0.292860,0.094093,0.437761,0.106110,0.056762,0.190334,0.916042,0.114529,0.186667,...,0.637724,0.334141,0.020429,0.184066,0.094357,0.778627,0.418502,0.030799,0.055459,0.052733
2,0.114651,0.192895,0.151723,0.388125,0.178486,0.063225,0.311401,0.934817,0.518635,0.342222,...,0.416398,0.337515,0.020875,0.250473,0.021118,0.828130,0.418502,0.027911,0.077430,0.041041
3,0.921666,0.088886,0.423663,0.570066,0.157089,0.063586,0.249114,0.930858,0.323706,0.253333,...,0.517208,0.516301,0.068177,0.173980,0.072054,0.817692,0.418502,0.030799,0.050685,0.071593
4,0.921134,0.088137,0.489140,0.527301,0.118082,0.059441,0.283522,0.929021,0.347827,0.226667,...,0.481356,0.439895,0.013378,0.191465,0.079407,0.805500,0.418502,0.059673,0.059898,0.012488


# Построение модели

In [6]:
import torch
from dataset import *
from utils import *
from models import *
import torch.nn as nn

In [11]:
k_val = 5

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 100

count_row_one_k = len(X) // k_val
mae_train_all, mae_val_all = [], []
for i in range(k_val):

  x_train_t = np.concatenate([X[:i * count_row_one_k], X[(i + 1) * count_row_one_k:]], axis=0)
  y_train_t = np.concatenate([y[:i * count_row_one_k], y[(i + 1) * count_row_one_k:]], axis=0)

  x_val_t = X[i * count_row_one_k: (i + 1) * count_row_one_k]
  y_val_t = y[i * count_row_one_k: (i + 1) * count_row_one_k]

  train_data_t = MyDataset(x_train_t, y_train_t)
  train_dataloader_t = DataLoader(train_data_t, batch_size=16, shuffle=False)

  test_data_t = MyDataset(x_val_t.to_numpy(), y_val_t.to_numpy())
  test_dataloader_t = DataLoader(test_data_t, batch_size=16, shuffle=False)

  model = MyModel(50, 64).to(device)

  criterion = nn.MSELoss()
  learning_rate = 0.0001

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  mae_train, mae_val = train_model(model, criterion, optimizer, epochs, train_dataloader_t, test_dataloader_t)

  mae_train_all.append(mae_train)
  mae_val_all.append(mae_val)

print('\nMean MAE on the train set k-value: '+ str(sum(mae_train_all) / len(mae_train_all)))
print('Mean MAE on the validation k-value: '+ str(sum(mae_val_all) / len(mae_val_all)))



Epoch: 0
Loss: tensor(0.0197, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.09109018154201969
Mean MAE on the test set: 0.08506813196417613


Epoch: 1
Loss: tensor(0.0088, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08522122903696952
Mean MAE on the test set: 0.08146936694780986


Epoch: 2
Loss: tensor(0.0098, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08158377711811374
Mean MAE on the test set: 0.08068123966073379


Epoch: 3
Loss: tensor(0.0167, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08045354644137044
Mean MAE on the test set: 0.07763298620016147


Epoch: 4
Loss: tensor(0.0097, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07776182438096692
Mean MAE on the test set: 0.07754192596826798


Epoch: 5
Loss: tensor(0.0077, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07724687743090815
Mean MAE on the test set: 0.07646803710705195


Epoch: 6
Loss: tensor(0.0097, grad_fn=<MseLossBackward0>)
Mean MAE on the train 

In [12]:
train_data = MyDataset(X.to_numpy(), y.to_numpy())
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

epochs = 100

model = MyModel(50, 64).to(device)

criterion = nn.MSELoss()
learning_rate = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
final_mae_train = train_model(model, criterion, optimizer, epochs, train_dataloader)

torch.save(model, 'model_ep100')
print(final_mae_train)



Epoch: 0
Loss: tensor(0.0129, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.10486218431684638


Epoch: 1
Loss: tensor(0.0120, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.09008546789307051


Epoch: 2
Loss: tensor(0.0334, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08539959031683175


Epoch: 3
Loss: tensor(0.0338, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08245383121961139


Epoch: 4
Loss: tensor(0.0141, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07951959656850661


Epoch: 5
Loss: tensor(0.0045, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07898712621451659


Epoch: 6
Loss: tensor(0.0205, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07771357716365182


Epoch: 7
Loss: tensor(0.0132, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07684743645663707


Epoch: 8
Loss: tensor(0.0041, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07618946186387478


Epoch: 9
Loss: tensor(0.0342, grad_